# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации о данных

In [104]:
import pandas as pd
solvency_statistics = pd.read_csv('/datasets/data.csv')
#print(solvency_statistics.head(5)) #распечатаем первые 5 строк таблицы
solvency_statistics.info() #запросим сводную информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Каждая строка содержит статистику платёжеспособности клиентов, включающую в себя информацию о наличии у заемщика детей, его семейном статусе, трудовом стаже и текущем статусе, размере ежемесячного дохода  По итогам действий, осущественных на первом шаге, можно выявить следующие недочеты полученных данных:
1. некоторые названия столбцов не вполне верно передают суть столбца и требуют корректировки в дальнейшем: children(например, при первом взгляде непонятно- речь идет о факте наличия детей или их количестве), dob_years, debt, total_income, purpose;
2. сразу же виден факт наличия пропущенных значений в двух столбцах - days_employed(трудовой стаж) и total_income(ежемесячный доход);
3. некоторые типы данных распознаны неправильно: days_employed распознан как float, хотя "в днях" предполагает тип int, total_income также распознан как float, хотя предполагался int;
4. присутствуют дубликаты, которые в том числе связаны с иным регистром и разными формулировками одного и того же.

Следующий этап анализа будет связан с корректировкой и устранением существующих проблем.

##  Предобработка данных

### Корректировка названий столбцов

In [105]:
solvency_statistics.set_axis (['number_of_children','days_employed','age_of_clients','educational_level','educational_level_id', 'family_status','family_status_id','gender', 'income_type','debt', 'monthly_income','purpose'], axis = 'columns', inplace = True)

In [106]:
solvency_statistics.columns

Index(['number_of_children', 'days_employed', 'age_of_clients',
       'educational_level', 'educational_level_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'monthly_income',
       'purpose'],
      dtype='object')

### Обработка пропусков

Второй шаг второго этапа анализа связан с поиском и обработкой пропущенных значений. В первую очередь установим, какие в данных есть пропуски, где они находятся и как отображаются.

In [107]:
solvency_statistics.isna().sum()

number_of_children         0
days_employed           2174
age_of_clients             0
educational_level          0
educational_level_id       0
family_status              0
family_status_id           0
gender                     0
income_type                0
debt                       0
monthly_income          2174
purpose                    0
dtype: int64

По результатам проведенных действий можно сделать следующий выводы:
1. значения, указывающие на пропущенные значения в данных, - NaN, что позволяет проводить с ними математические операции;
2. в нашем датафрейме переменные, в которых присутствуют пропущенные значения, являются количественными(общий трудовой стаж, ежемесячный доход), соответственно, для решения проблемы пропущенных значений стоит обратиться к характерным значениям выборки. 

In [108]:
print('Количество пропусков в колонке "days_employed" до заполнения:', solvency_statistics['days_employed'].isna().sum())
groups = solvency_statistics.groupby(['educational_level','income_type'])['days_employed'].transform('median')
solvency_statistics['days_employed'] = solvency_statistics['days_employed'].fillna(groups)
print('Количество пропусков в колонке "days_employed" после заполнения:', solvency_statistics['days_employed'].isna().sum())

Количество пропусков в колонке "days_employed" до заполнения: 2174
Количество пропусков в колонке "days_employed" после заполнения: 0


In [109]:
print('Количество пропусков в колонке "monthly_income" до заполнения:', solvency_statistics['monthly_income'].isna().sum())
groups = solvency_statistics.groupby(['educational_level','income_type'])['monthly_income'].transform('median')
solvency_statistics['monthly_income'] = solvency_statistics['monthly_income'].fillna(groups)
print('Количество пропусков в колонке "monthly_income" после заполнения:', solvency_statistics['monthly_income'].isna().sum())

Количество пропусков в колонке "monthly_income" до заполнения: 2174
Количество пропусков в колонке "monthly_income" после заполнения: 0


Следующий этап предобработки данных - обработка нулевых значений, обнаруженных в колонке "возраст клиента", и их замена на среднее значение(в случае возраста мы можем взять просто среднее, а не медиану).

In [118]:
solvency_statistics['age_of_clients'].value_counts() # проверим возраст на наличие нулевых значений
age_avg = (solvency_statistics.groupby('income_type').agg({'age_of_clients':'median'}).rename(columns = {'age_of_clients': 'median_age'}))
data = solvency_statistics.merge(age_avg, on = ['income_type'])
data.loc[data['age_of_clients'] == 0, 'age_of_clients'] = data.loc[data['age_of_clients'] == 0, 'median_age'] #заменим нулевые значения на медианные
print(data[data['age_of_clients'] == 0])

Empty DataFrame
Columns: [number_of_children, days_employed, age_of_clients, educational_level, educational_level_id, family_status, family_status_id, gender, income_type, debt, monthly_income, purpose, median_age]
Index: []


**Вывод**

По итогам второго шага были обнаружены 2174 пропущенных значений типа 'NaN' в столбцах с категориальными переменными "days_employed" и "monthly_income". Среди возможных причин появления пропусков данных - их отсутствие в исходной базе данных(например, данные по доходам трудно восстановить, если человек работает неофициально), возможно также,что данные были неправильно распознаны системой. Не стоит исключать и банальных ошибок, связанных с невнимательностью составителя базы. Соотвественно, пропуски были заполнены на основе стратегии нахождения характерных значений выборки, а именно медианы (для количественных переменных это возможно). Для достижения результата была выбрана стратегия заполнения пропусков с группировкой по нескольким переменным и применением метода transform. Кроме того, были обнаружены нулевые значения в колонке "возраст", которые были заменены с помощью медианы в зависимости от типа занятости. 


### Замена типа данных

Третий шаг исследования направлен на приведение данных к целочисленному типу для большего удобства дальнейшей работы и восприятия информации.

In [120]:
data.loc[solvency_statistics['days_employed'] > 25550, 'days_employed'] = data.loc[solvency_statistics['days_employed'] > 25550, 'days_employed'] / 24
data['age_of_clients'] = data['age_of_clients'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['monthly_income'] = data['monthly_income'].astype('int')
del data['median_age']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
number_of_children      21525 non-null int64
days_employed           21525 non-null int64
age_of_clients          21525 non-null int64
educational_level       21525 non-null object
educational_level_id    21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
monthly_income          21525 non-null int64
purpose                 21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [121]:
data['days_employed'] = data['days_employed'].abs() #избавимся от отрицательных значений
print(data['days_employed'].head()) #проверим факт ликвидации минуса

0    8437
1    4024
2    5623
3    4124
4       6
Name: days_employed, dtype: int64


**Вывод**

Данный шаг был направлен на преобразование типов данных от вещественного типа к численному: и колонка 'days_employed', и колонка 'monthly_income' первоначально были распознаны системой как float, что затрудняло дальнейшую работу с данными и делало неудобным как их восприятие, так и анализ. Для изменения данных был выбран метод astype(), позволяющий переводить значения в нужный нам тип. Метод to_numeric() нам не подходил, так как при переводе все числа бы имели тип данных float, что нам не подходит. Кроме того, путем применения аргумента abs() удалось избавиться от отрицательных значений столбца 'days_employed', которые создавали путанницу в данных.

### Обработка дубликатов

Четвертый шаг анализа необходим для поиска и удаления дублирующих значений, искажающих данные.  

In [122]:
print ('Дубликатов в таблице до обработки данных:', data.duplicated().sum()) # проведем общий подсчет дубликато в таблице

Дубликатов в таблице до обработки данных: 38


In [123]:
#print (data['educational_level'].value_counts()) # в первую очередь проверим ункальные значения в строковых данных
data['educational_level'] = data['educational_level'].str.lower() # перевод значений в нижний регистр
print(data['educational_level'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: educational_level, dtype: int64


In [124]:
#print (data['family_status'].value_counts()) # в первую очередь проверим ункальные значения в строковых данных
data['family_status'] = data['family_status'].str.lower() # перевод значений в нижний регист
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [125]:
#print (data['number_of_children'].value_counts()) #  проверим ункальные значения в числовых данных
data['number_of_children'] = data['number_of_children'].abs() # возьмем значения по модулю для ликвидации минуса
print (data['number_of_children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: number_of_children, dtype: int64


In [127]:
print('Дубликатов в таблице после обработки данных:', data.duplicated().sum())

Дубликатов в таблице после обработки данных: 38


In [128]:
data = data.drop_duplicates() # удалим дубликаты
print(data.info()) # проверим тип данных

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21487 entries, 0 to 21524
Data columns (total 12 columns):
number_of_children      21487 non-null int64
days_employed           21487 non-null int64
age_of_clients          21487 non-null int64
educational_level       21487 non-null object
educational_level_id    21487 non-null int64
family_status           21487 non-null object
family_status_id        21487 non-null int64
gender                  21487 non-null object
income_type             21487 non-null object
debt                    21487 non-null int64
monthly_income          21487 non-null int64
purpose                 21487 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
None


**Вывод**

На этапе поиска и обработки дубликатов в данных, во-первых, был осуществлен ручной поиск дубликатов методом duplicated() в сочетании с методом sum() для формирования первичной картины наличия дубликатов, а также метод value.counts(), позволяющий обнаружить конкретные столбцы с наличием дубликатов. Далее был осуществлен ручной поиск дубликатов с учетом регистра в строковых данных - уровне образования клиента и его семейном положении с помощью вызова метода str.lower(). После данных преобразований все дубликаты были удалены методом drop_duplicates(). 

### Лемматизация

Так как колонка с перечнем целей кредита сдержит в себе множество слов, записанных в разных формах, для дальнейшей работы необходимо провести лемматизацию колонки, на что и направлен пятый шаг анализа.

In [129]:
from pymystem3 import Mystem # осуществим лемматизацию с помощью библиотеки  pymystem3
m = Mystem()
lemmas = m.lemmatize(' '.join(data['purpose']))
from collections import Counter
lemmas_count = Counter(lemmas)
lemmas_count_final = pd.DataFrame.from_dict(lemmas_count, orient='index').reset_index()
lemmas_count_final = lemmas_count_final.rename(columns={'index':'word', 0:'count'})# выведем список "слово-кол-во упоминаний"
print(lemmas_count_final.sort_values(by = 'count', ascending = False))

              word  count
1                   55108
13    недвижимость   6356
0          покупка   5903
2            жилье   4463
4       автомобиль   4309
6      образование   4016
23               с   2918
19        операция   2606
9          свадьба   2343
16            свой   2233
7               на   2231
14   строительство   1879
26         высокий   1374
21       получение   1315
12    коммерческий   1313
10             для   1292
27           жилой   1231
24          сделка    941
5   дополнительный    908
22      заниматься    905
17       подержать    845
8       проведение    776
18         сыграть    772
28           сдача    652
11           семья    640
15     собственный    635
20              со    629
29          ремонт    607
3     приобретение    462
25      профильный    436
30     подержанный    119
31              \n      1


**Вывод**

С помощью библиотеки  pymystem3 была осуществлена лемматизация колонки, содержащей информацию о целях кредитования. В результате приведения каждого слова к словарной форме было выделено пять основных категорий, по которым в дальнейшем будет осуществлена категоризация данных: недвижимость, жилье, автомобиль, образование, свадьба. Несмотря на большое количество упоминаний, существительные "покупка" и "операция" не были включены в перечень, так как чаще всего они употребляются в имеющихся данных в связке с недвижимостью.

### Категоризация данных

Для того, чтобы данные можно было легко воспринимать, проведем категоризацию данных.

In [130]:
def purpose_category(purpose):
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'не найдено'
data['purpose'] = data['purpose'].apply(purpose_category)
print (data['purpose'].value_counts())

недвижимость    10819
автомобиль       4309
образование      4016
свадьба          2343
Name: purpose, dtype: int64


In [144]:
def purpose_plus_debt(row):
    purpose = row['purpose']
    debt = row['debt']
    if purpose in data['purpose'].unique() and debt  == 0:
        return 'кредит возвращен'
    else:    
        return 'кредит не возвращен'
data['purpose_plus_debt'] = data.apply(purpose_plus_debt, axis=1)

In [131]:
print ('Дубликатов в таблице после категоризации целей:', data.duplicated().sum())
data= data.drop_duplicates()
print('Дубликатов в таблице после удаления:',data.duplicated().sum())

Дубликатов в таблице после категоризации целей: 232
Дубликатов в таблице после удаления: 0


В результате категоризации целей кредита удалось выделить основные категории, описывающие цели клиентов и классифицировать их по наличию/отсутствию задолженности(факту возврата/невозврата кредита). Стоит сказать, что категории "жилье" и "недвижимость" были объединены в "недвижимость", так как разделение на личную и коммерческую недвижимость видится непринципиальным для цели исследования. Поскольку на этапе поиска и удаления дубликатов полной картины по целям не было, необходимо было проверить и удалить появившиейся дубликаты. 

In [132]:
def classifying_children(row):
    row_value = row['number_of_children']
    if row_value == 0:
        return 'нет детей'
    elif row_value == 1:
        return 'один ребенок'
    elif row_value >= 2 :
        return 'два и более ребенка'
data['number_of_children'] = data.apply(classifying_children, axis=1)
data['number_of_children'].value_counts()

нет детей              13939
один ребенок            4817
два и более ребенка     2499
Name: number_of_children, dtype: int64

In [136]:
def children_plus_debt(row):
    children = row['number_of_children']
    debt = row['debt']
    if children in data['number_of_children'].unique() and debt  == 0:
        return 'кредит возвращен'
    else:    
        return 'кредит не возвращен'
data['children_plus_debt'] = data.apply(children_plus_debt, axis=1)

In [138]:
def status_plus_debt(row):
    family_status = row['family_status']
    debt = row['debt']
    if family_status in data['family_status'].unique() and debt  == 0:
        return 'кредит возвращен'
    else:    
        return 'кредит не возвращен'
data['status_plus_debt'] = data.apply(status_plus_debt, axis=1)

In [140]:
def analyzing_income(row):
    row_value = row['monthly_income']
    if row_value < 60000:
        return 'низкий уровень дохода'
    elif row_value >= 60000 and row_value <= 150000 :
        return 'средний уровень дохода'
    elif row_value > 150000:
        return 'высокий уровень дохода'
data['monthly_income'] = data.apply(analyzing_income, axis=1)
data['monthly_income'].value_counts()

средний уровень дохода    10523
высокий уровень дохода     9926
низкий уровень дохода       806
Name: monthly_income, dtype: int64

In [141]:
def income_plus_debt(row):
    income = row['monthly_income']
    debt = row['debt']
    if income  in data['monthly_income'].unique() and debt  == 0:
        return 'кредит возвращен'
    else:    
        return 'кредит не возвращен'
data['income_plus_debt'] = data.apply(income_plus_debt, axis=1)

**Вывод**

Этап категоризации данных позволил сгруппировать данные по укрупненным группам для дальнейшего анализа и стал прмежуточным шагом к финальным ответам на вопросы.

## Анализ данных

Анализируя данные для ответа на поставленные вопросы, будем в первую очередь обращать внимание на данные по возврату кредита не в срок.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [137]:
data_pivot_children = data.pivot_table(index=['number_of_children'], columns='children_plus_debt', values='debt', aggfunc='count')
data_pivot_children['доля невозвратов в общем кол-ве кредитов, %'] = (data_pivot_children['кредит не возвращен'] / (data_pivot_children['кредит не возвращен'] + data_pivot_children['кредит возвращен'])) *100
data_pivot_children.sort_values(by = 'доля невозвратов в общем кол-ве кредитов, %', ascending = False)

children_plus_debt,кредит возвращен,кредит не возвращен,"доля невозвратов в общем кол-ве кредитов, %"
number_of_children,,,
два и более ребенка,2266,233,9.323729
один ребенок,4372,445,9.238115
нет детей,12877,1062,7.618911


**Вывод**

Как можно увидеть из приведенных данных, клиенты, имеющие детей, имеют задолженность по кредиту в большем количестве случаев (9.27 -9.60%), чем клиенты, не имеющие детей (доля невозврата - 7.67%). Это может быть объяснено тем, что большее количество детей требует больших финансовых "вливаний", что часто способствует нехватке денег для возврата кредита. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [139]:
data_pivot_status = data.pivot_table(index=['family_status'], columns='status_plus_debt', values='debt', aggfunc='count')
data_pivot_status['доля невозвратов в общем кол-ве кредитов, %'] = (data_pivot_status['кредит не возвращен'] / (data_pivot_status['кредит не возвращен'] + data_pivot_status['кредит возвращен'])) *100
data_pivot_status.sort_values(by = 'доля невозвратов в общем кол-ве кредитов, %', ascending = False)

status_plus_debt,кредит возвращен,кредит не возвращен,"доля невозвратов в общем кол-ве кредитов, %"
family_status,,,
не женат / не замужем,2523,274,9.796210
гражданский брак,3758,388,9.358418
женат / замужем,11237,930,7.643626
в разводе,1109,85,7.118928
вдовец / вдова,888,63,6.624606


**Вывод**

Как видно из приведенной таблицы, наиболее аккуратны в возврате кредита клиенты, имеющие семью (доля несвоевременных возвратов - 7.7%), люди, находящиеся в разводе (7.12%), и люди, потерявшие мужа/жену (6.68%),- при этом стоит отметить, что выборка людей в статусе "вдовец/вдова" у нас достатчно небольшая, что не позволяет считать вывод по ним окончательно верным. Люди, живущие одни или не связанные официальными отншениями, менее аккуратны и обязательны в выплате кредита (9.84% и 9.4%, соответственно). Среди возможных причин - наличие некоторого "контроля" со стороны мужа/жены и, соответственно, второго "источника дохода", что, во-первых, упрощает выплату средств и, во-вторых, позволяет осуществлять ее своевременно. Соответственно, люди, живущие одни или "условно" не одни, обладают более низким уровнем ответственности и меньшей свободой действий в случае возможно трудной жизненной ситуации.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [142]:
data_pivot_income = data.pivot_table(index=['monthly_income'], columns='income_plus_debt', values='debt', aggfunc='count')
data_pivot_income['доля невозвратов в общем кол-ве кредитов, %'] = (data_pivot_income['кредит не возвращен'] / (data_pivot_income['кредит не возвращен'] + data_pivot_income['кредит возвращен'])) *100
data_pivot_income.sort_values(by = 'доля невозвратов в общем кол-ве кредитов, %', ascending = False)

income_plus_debt,кредит возвращен,кредит не возвращен,"доля невозвратов в общем кол-ве кредитов, %"
monthly_income,,,
средний уровень дохода,9602,921,8.752257
высокий уровень дохода,9156,770,7.757405
низкий уровень дохода,757,49,6.079404


**Вывод**

В большей степени люди, имеющие условно "средние" и "высокие" доходы, склонны задерживать выплату кредита, чем люди с низкими доходами. Возможно, это связано с тем, что для людей со средними и высокими доходами кредит - это способ добрать небольшую сумму для покупки какой-то статусной вещи, в то время как для людей с низкими доходами кредит - это важное событие в жизни, кторому придается большее значение.

- Как разные цели кредита влияют на его возврат в срок?

In [145]:
data_pivot_purpose = data.pivot_table(index=['purpose'], columns='purpose_plus_debt', values='debt', aggfunc='count')
data_pivot_purpose['доля невозвратов в общем кол-ве кредитов, %'] = (data_pivot_purpose['кредит не возвращен'] / (data_pivot_purpose['кредит не возвращен'] + data_pivot_purpose['кредит возвращен'])) *100
data_pivot_purpose.sort_values(by = 'доля невозвратов в общем кол-ве кредитов, %', ascending = False)

purpose_plus_debt,кредит возвращен,кредит не возвращен,"доля невозвратов в общем кол-ве кредитов, %"
purpose,,,
автомобиль,3879,403,9.411490
образование,3613,370,9.289480
свадьба,2139,186,8.000000
недвижимость,9884,781,7.323019


**Вывод**

На основе проведенной категоризации данных, содержащих информацию о целях кредита и количестве возвращенных и невозвращенных кредитов, можно сделать вывод о том, что, несмотря на то, что абсолютные значения говорят нам о том, что больше всего невозратных кредитов в категории "недвижимость", а меньше всего - в категории "свадьба", относительные значения колонки "долля невозвратных кредитов в общем количестве кредитов", взятых для определенной цели, отличаются совсем на небольшое количество процентных пунктов. Тем не менее, можно сказать, что кредиты, взятые для покупки личной или коммерческой недвижимости, в большинстве случаев возвращают в срок (доля невозвратов - 7.3%), в то время как кредиты, взятые для покупки автомобиля, возвращают в срок реже (доля невозвратов - 9.4%). Среди возможных объяснений этой тенденции можно выделить большую "взвешенность" решения по приобретению недвижимости - как правило, покупка квартиры - это семейное  и коллективное решение, как и приобретение коммерческой недвижимости, возможно, для открытия собственного бизнеса или сдачи в аренду. В свою очередь покупка автомобиля в кредит - не всегда твердое решение, кредит на покупку автомобиля зачастую берут люди, которые хотят мнимой статусности  в глазах окружающих, поэтому он не всегда возвращается своевременно.

## Общий вывод

В результате проведенного анализа были изучены несколько факторов, влияющих на своевременность возврата кредита. Во-первых, было выяснено, что количество детей увеличивает вероятность невозврата кредита в нужный срок, в то время как их отсутствие - уменьшает. Во-вторых, клиенты, находящиеся в "официальных" отношениях (женат/замужем, в разводе), склонны возвращать кредит в срок чаще, чем люди условно "свободные". В-третьих, кредит, который взят с целью покупки личной или коммерческой недвижимости, будет возвращен в срок с большей вероятностью, чем кредит на покупку автомобиля или получение образования. Наконец, люди с средними и высокими доходами менее аккуратны в выплате кредита, чем люди с низкими доходами.